# 02. 데이터 품질 분석 및 비즈니스 인사이트
## Olist 프로젝트 데이터 검증 보고서

**분석 목적:**
- 전체 데이터셋의 결측치 및 이상치 식별
- 주요 수치형 변수의 기술통계량 확인
- 데이터 구조적 특징을 통한 비즈니스 인사이트 도출

**작성일:** 2026-01-17  
**작성자:** Anti-Gravity

---
## 1. 환경 설정 및 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# 프로젝트 루트 경로 추가 (src 모듈 임포트용)
sys.path.append(os.path.abspath('..'))

from src.data_loader import load_all_tables
from src.utils import set_korean_font

# 한글 폰트 설정
set_korean_font()

# 데이터 로드
tables = load_all_tables(verbose=True)

---
## 2. 도메인 지식: 주문 처리 프로세스 이해

분석에 앞서 주요 시간 데이터의 의미를 정의합니다.

1. **`order_approved_at`**: 결제 승인 시점. 결측 시 결제 실패 혹은 입금 전 단계 의미.
2. **`order_delivered_carrier_date`**: 물류사 인계 시점. **판매자의 속도**를 측정하는 핵심 지표.
3. **`order_delivered_customer_date`**: 최종 배송 완료일. **물류사(Carrier)의 역량**과 연관.

---
## 3. 테이블별 품질 분석 및 기술통계

In [ ]:
def analyze_table_quality(name, df):
    print(f"\n{'='*60}")
    print(f"📊 TABLE: {name.upper()}")
    print(f"{'='*60}")
    
    # 1. 결측치 확인
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if not missing.empty:
        print("\n[1. 결측치 현황]")
        display(pd.DataFrame({'Missing': missing, 'Ratio(%)': (missing/len(df)*100).round(2)}))
    else:
        print("\n✅ 결측치 없음")
        
    # 2. 기술통계
    num_df = df.select_dtypes(include=['number'])
    if not num_df.empty:
        print("\n[2. 기술통계량]")
        display(num_df.describe().T)
        
        # 3. 이상치 시각화
        plot_cols = [c for c in num_df.columns if 'zip' not in c.lower() and c != 'payment_sequential']
        if plot_cols:
            print(f"\n[3. 이상치 분석 - {len(plot_cols)}개 컬럼]")
            fig, axes = plt.subplots(1, len(plot_cols), figsize=(max(4*len(plot_cols), 12), 5))
            if len(plot_cols) == 1: axes = [axes]
            for i, col in enumerate(plot_cols):
                sns.boxplot(y=df[col].dropna(), ax=axes[i], color='skyblue')
                axes[i].set_title(col)
            plt.tight_layout()
            plt.show()

for name, df in tables.items():
    if df is not None:
        analyze_table_quality(name, df)

---
## 4. 핵심 분석 결과 및 인사이트

### 📍 4.1 배송 및 운영 (Operations)
- **Orders 테이블의 결측치(2.98%):** 배송이 완료되지 않은 '진행 중' 혹은 '실패' 주문입니다. 배송 지연 분석 시 이 데이터 집단의 성격을 먼저 정의해야 합니다.
- **배송비의 양극화:** `freight_value`의 최대치가 400헤알을 넘는 것은 브라질의 광활한 지리적 제약을 상징합니다.

### 📍 4.2 결제 및 매출 (Financials)
- **할부 지향적 소비:** 최대 24개월의 할부는 고가 제품군에 대한 높은 진입장벽을 낮추는 핵심 전략입니다.
- **GMV 롱테일 구조:** 소수의 고액 결제자가 전체 매출에서 차지하는 비중이 큽니다.

### 📍 4.3 데이터 정제 시 주의사항
- **Geo 데이터 오류:** 위경도 값 중 브라질 범위를 벗어나는 데이터(이상치)의 필터링이 선행되어야 합니다.
- **리뷰 텍스트 부족:** 60% 이상의 고객은 별점만 남깁니다. 감성 분석 시 텍스트 데이터의 대표성을 고려해야 합니다.

---
## 5. 다음 단계
- [ ] 분석 대상 주문 데이터셋 확정 (승인 완료 건 중심)
- [ ] 지역권별 배송비 및 배송 기간 상관분석